# Attention 메커니즘 구현하기

## 3.1 긴 시퀀스 모델링의 문제점
- Attention 이전의 문제점
    - RNN 이전 스텝의 출력이 현재 스텝의 입력으로 사용되는 신경망
    - 핵심 idea는 encoder가 전체 입력 텍스트를 하나의 은닉 상태로 처리
    - decoder가 이 은닉 상태를 받아 출력을 생성
    - RNN의 제약사항은 decoder가 encoder 이전의 은닉 상태를 참조할 수 없다는 것
    - **즉, 맥락을 놓칠 수 있다.**

## 3.2 Attention 메커니즘으로 데이터 의존성 포착하기
- Bahdanau attention은 RNN을 수정하여 decoding 단계마다 decoder가 선택적으로 입력 시퀀스의 서로 다른 부분을 참조할 수 있다.
- self Attention은 시퀀스의 표현을 계산할 때 입력 시퀀스에 있는 각 위치가 동일 시퀀스에 있는 다른 모든 위치와의 관련성을 고려하거나 주의를 기울일 수 있다.

## self Attention으로 입력의 서로 다른 부분에 주의 기울이기
- self는 하나의 입력 시퀀스에 있는 **서로 다른 위치의 원소 사이에 어텐션 가중치를 계산**하는 방식을 의미
- self Attention의 목표
    - 다른 모든 입력 원소의 정보를 조합하여
    - 각각의 입력 원소에 대한 문맥 벡터를 계산하는 것

### 3.3.1 훈련 가능한 가중치가 없는 간단한 self Attention 메커니즘
- "Your journey starts with one step"
- x는 특정 토큰을 표현하는 d 차원 임베딩 벡터다
    - "Your" [0.4][0.1][0.8] (예시는 3차원 벡터)
    - 문맥 벡터인 z는 x1 ~ xt(여기서 t는 상수) 사이의 정보를 담은 임베딩이다.
    - 문맥 벡터는 입력 시퀀스에 있는 모든 원소의 정보를 통합해 각 원소의 표현을 풍부하게 만드는 것이 목적이다.

다음은 3차원 벡터로 임베딩된 입력 시퀀스가 있다고 가정

In [48]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89],    # Your
     [0.55, 0.87, 0.66],    # journey
     [0.57, 0.85, 0.64],    # starts
     [0.22, 0.58, 0.33],    # with
     [0.77, 0.25, 0.10],    # one
     [0.05, 0.80, 0.55]]    # step
)

- self Attention 구현의 첫 단계는 attention score를 계산하는 것이다.
- attention score는 쿼리 x와 다른 모든 입력 토큰 사이의 점곱으로 결정한다.

In [49]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


> [!note]
> - 점곱은 두 벡터를 원소끼리 곱한 다음 모두 더하는 방법이다.
> - 점곱은 하나의 유사도 척도로 볼 수 있다. (두 벡터가 얼마나 가까이 놓여 있는지 정량화할 수 있기 때문)
> - 점곱이 높을수록 두 원소 사이의 유사도와 attention score가 높다.

In [50]:
res = 0
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- attention score 정규화는 attention 가중치의 합이 1이 되도록 하기 위해서다.
- 정규화는 해석하기 용이하고 LLM 훈련 시 안정성을 유지하는 데 도움이 된다.

In [51]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("attention weights: ", attn_weights_2_tmp)
print("합: ", attn_weights_2_tmp.sum())

attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
합:  tensor(1.0000)


- attention score를 softmax로 정규화
- softmax는 attention weights 합이 1이다.

In [52]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("attention weights: ", attn_weights_2)
print("합: ", attn_weights_2.sum())

attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합:  tensor(1.)


- 임베딩된 입력 토큰 x와 각 토큰에 해당하는 attention 가중치를 곱한 후 모두 더해서 문맥 벡터 z 계산

In [53]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 모든 입력 토큰에 대해 attention 가중치 계산하기
- 모든 문맥 벡터를 계산하기 위해 코드 수정

In [54]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 행렬 곱셈을 사용한 결과

In [55]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 모든 합이 1이 되도록 정규화

In [56]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- dim=-1로 하면 softmax가 attn_scores tensor의 마지막 차원을 따라 정규화를 수행
- 예) 2차원 tensor라면, 열을 따라서 각 행의 값을 모두 더해 1이 되도록 정규화

In [57]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

print("두 번째 행의 합: ", row_2_sum)
print("모든 행의 합: ", attn_weights.sum(dim=-1))

두 번째 행의 합:  1.0
모든 행의 합:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 어텐션 가중치와 입력을 행렬 곱셈하여 모든 문맥 벡터 계산

In [58]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [59]:
print("앞서 계산한 두 번째 문맥 벡터: ", context_vec_2)

앞서 계산한 두 번째 문맥 벡터:  tensor([0.4419, 0.6515, 0.5683])


## 3.4 훈련 가능한 가중치를 가진 self Attention 구현하기
- self Attention 메커니즘을 scaled dot-product attention 이라고 부른다.
- 훈련 가능한 가중치를 가진 self Attention 메커니즘은 특정 입력 원소에 대한 입력 벡터의 가중치 합으로 문백 벡터를 계산한다.
    - self Attention과 가장 큰 차이점은 모델 훈련 과정에서 업데이트되는 가중치 행렬이 추가된 것

### 3.4.1 단계별로 Attention 가중치 계산하기
- 훈련 가능한 가중치 행렬 3개 (Wq, Wk, Wv)를 추가하여 self Attention 메커니즘을 단계별로 구현
- 예시를 위해 먼저 하나의 문맥 벡터를 계산 하고 다중에 모든 문맥 벡터를 계산

In [60]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

- 출력을 간단하기 만들려고 requires_grad=False 모델 훈련 시에는 True로 지정하여 가중치 행렬을 업데이트해야 한다.

In [61]:
torch.manual_seed(123)

w_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [62]:
query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value

print(query_2)

tensor([0.4306, 1.4551])


In [63]:
# 6개의 입력 토큰을 3차원에서 2차원 임베딩 공간에 투영

keys = inputs @ w_key
values = inputs @ w_value

print("keys.shape: ", keys.shape)
print("values.shape: ", values.shape)

keys.shape:  torch.Size([6, 2])
values.shape:  torch.Size([6, 2])


- 어텐션 점수 계산

In [64]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)

print(attn_score_22)

tensor(1.8524)


- 행렬 곱으로 일반화하여 모든 어텐션 점수 계산

In [65]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


- attention score에서 attention 가중치 구하기

In [66]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)

print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


> [!note]
> 스케일드 점곱 어텐션을 사용하는 이유
> - 임베딩 차원의 크기로 정규화를 하는 이유
>    - 그레이디언트가 작아지는 것을 피하기 위해 (훈련 성능을 높이기 위해)
>    - 임베팅 차원이 크면 역전파 과정에서 매우 작은 그레이디언트르 생성
> - 임베딩 차원의 제곱그능로 나누기 때문에 self Attention 메커니즘을 스케일드 점곱 어텐션이라고 부른다.

- 입력 벡터에 대한 가중치 합으로 문맥 벡터를 계산했을 때 와 비슷하게
- value 벡터에 대한 가중치 합으로 문맥 벡터를 계산한다.

In [67]:
context_vec_2 = attn_weights_2 @ values

print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 self Attention 파이썬 클래스 구현하기

In [68]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()

        self.w_query = nn.Parameter(torch.rand(d_in, d_out))
        self.w_key = nn.Parameter(torch.rand(d_in, d_out))
        self.w_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.w_key
        queries = x @ self.w_query
        values = x @ self.w_value

        attn_scores = queries @ keys.T  # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim=-1
        )

        context_vec = attn_weights @ values

        return context_vec

In [69]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)

print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


- SelfAttention_v1 구현을 pytorch의 nn.Linear 층을 사용하도록 개선할 수 있다.
- nn.Linear는 bias unit을 사용하지 않는 경우 행렬 곱셈과 동일한 연산을 수행
- nn.Linear 층은 최적화된 가중치 초기화 방법을 사용할 수 있어 모델 훈련을 안정적이고 효율적으로 만든다.

In [70]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()

        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.w_key(x)
        queries = self.w_query(x)
        values = self.w_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim=-1
        )

        context_vec = attn_weights @ values

        return context_vec

In [72]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)

print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## 3.5 코잘 attention으로 미래 단어를 감추기
- masked attention 이라 부르는 코잘 attention은 self attention의 특별한 형태
    - 주어진 토큰으로 attention score를 계산할 때 시퀀스의 이전 입력과 현재 입력만 참조하도록 한다.

### 3.5.1 코잘 attention mask 적용하기
- softmax 함수를 사용해 attention weight를 계산한다

In [75]:
queries = sa_v2.w_query(inputs)
keys = sa_v2.w_key(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)

print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- pytorch tril 함수로 주대각선 위의 값이 0인 마스크를 만든다.

In [76]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))

print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 마스크와 attention weight를 곱해서 주대각선 위의 값을 0으로 만든다.

In [77]:
masked_simple = attn_weights * mask_simple

print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 가중치의 합이 1이 되도록 다시 정규화 한다.
- 각 행의 합으로 행의 원소를 나누면 된다.

In [78]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums

print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- softmax 함수는 입력을 확률 분포로 변환한다.
- 한 행에 음의 무한대 값이 있으면 softmax는 해당 값을 0으로 만든다.
- 주대각선 위의 값이 1인 mask를 만들고 1을 -inf으로 바꾸는 식으로 더 효율적인 masking 기법을 구현할 수 있다.

In [82]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

masked = attn_scores.masked_fill(mask.bool(), -torch.inf)

print(masked)

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])
tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [80]:
attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=1)

print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


> [!note]
> - tril()은 하삼각 행렬을 반환한다. (주대각선 위의 값을 0)
> - tiru()는 상삼각 행렬을 반환한다. (주대각선 아래의 값을 0으로)
> - diagonal 값이 0일 때, 주대각선의 원소를 포함하여 반환
> - 1일 때, 주대각선의 원소도 0으로 만듬
> - bool()은 tensor 값을 bool type으로 변환
> - masked_fill()은 첫 번째 매개변수로 전달된 bool mask가 1인 위치에 두 번째 매개변수 값을 채운다.